<a href="https://colab.research.google.com/github/ipeirotis-org/datasets/blob/main/NYPD_Complaint/NYPD.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## NYPD Dataset

Dataset description at
https://data.cityofnewyork.us/Public-Safety/NYPD-Complaint-Data-Historic/qgea-i56i



| Column | Description |
|--------|-------------------|
| CMPLNT_NUM |  Randomly generated persistent ID for each complaint  |  
| ADDR_PCT_CD |  The precinct in which the incident occurred |  
| BORO |  The name of the borough in which the incident occurred |  
| CMPLNT_FR_DT |  Exact date of occurrence for the reported event (or starting date of occurrence, if CMPLNT_TO_DT exists) |  
| CMPLNT_FR_TM |  Exact time of occurrence for the reported event (or starting time of occurrence, if CMPLNT_TO_TM exists) |  
| CMPLNT_TO_DT |  Ending date of occurrence for the reported event, if exact time of occurrence is unknown |  
| CMPLNT_TO_TM |  Ending time of occurrence for the reported event, if exact time of occurrence is unknown |  
| CRM_ATPT_CPTD_CD |  Indicator of whether crime was successfully completed or attempted, but failed or was interrupted prematurely |  
| HADEVELOPT |  Name of NYCHA housing development of occurrence, if applicable |  
| HOUSING_PSA |  Development Level Code |  
| JURISDICTION_CODE |  Jurisdiction responsible for incident. Either internal, like Police(0), Transit(1), and Housing(2); or external(3), like Correction, Port Authority, etc. |  
| JURIS_DESC |  Description of the jurisdiction code |  
| KY_CD |  Three digit offense classification code |  
| LAW_CAT_CD |  Level of offense: felony, misdemeanor, violation  |  
| LOC_OF_OCCUR_DESC |  Specific location of occurrence in or around the premises; inside, opposite of, front of, rear of |  
| OFNS_DESC |  Description of offense corresponding with key code |  
| PARKS_NM |  Name of NYC park, playground or greenspace of occurrence, if applicable (state parks are not included) |  
| PATROL_BORO |  The name of the patrol borough in which the incident occurred |  
| PD_CD |  Three digit internal classification code (more granular than Key Code) |  
| PD_DESC |  Description of internal classification corresponding with PD code (more granular than Offense Description) |  
| PREM_TYP_DESC |  Specific description of premises; grocery store, residence, street, etc. |  
| RPT_DT |  Date event was reported to police  |  
| STATION_NAME |  Transit station name |  
| SUSP_AGE_GROUP |  Suspect’s Age Group |  
| SUSP_RACE |  Suspect’s Race Description |  
| SUSP_SEX |  Suspect’s Sex Description |  
| TRANSIT_DISTRICT |  Transit district in which the offense occurred. |  
| VIC_AGE_GROUP |  Victim’s Age Group |  
| VIC_RACE |  Victim’s Race Description |  
| VIC_SEX |  Victim’s Sex Description |  
| X_COORD_CD |  X-coordinate for New York State Plane Coordinate System, Long Island Zone, NAD 83, units feet (FIPS 3104) |  
| Y_COORD_CD |  Y-coordinate for New York State Plane Coordinate System, Long Island Zone, NAD 83, units feet (FIPS 3104) |  
| Latitude |  Midblock Latitude coordinate for Global Coordinate System, WGS 1984, decimal degrees (EPSG 4326)  |  
| Longitude |  Midblock Longitude coordinate for Global Coordinate System, WGS 1984, decimal degrees (EPSG 4326) |


In [1]:
!pip install -q google-cloud-secret-manager

from google.colab import auth
auth.authenticate_user()

from google.cloud import secretmanager

def access_secret_version(project_id, secret_id, version_id):
    """
    Access the payload of the given secret version and return it.

    Args:
        project_id (str): Google Cloud project ID.
        secret_id (str): ID of the secret to access.
        version_id (str): ID of the version to access.
    Returns:
        str: The secret version's payload, or None if
        the version does not exist.
    """
    client = secretmanager.SecretManagerServiceClient()
    name = f"projects/{project_id}/secrets/{secret_id}/versions/{version_id}"
    response = client.access_secret_version(request={"name": name})
    return response.payload.data.decode("UTF-8")


mysql_pass = access_secret_version("nyu-datasets", "MYSQL_PASSWORD", "latest")

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 180.7/180.7 kB 1.5 MB/s eta 0:00:00


In [2]:
import pandas as pd
import numpy as np

In [3]:
# We load everything as an object/string, because some data types (e.g., some IDs)
# are recognized as decimals, and it is a mess to restore them back
# So we will do all the conversions ourselves later on

# From https://data.cityofnewyork.us/Public-Safety/NYPD-Complaint-Data-Historic/qgea-i56i/data
!curl 'https://data.cityofnewyork.us/api/views/qgea-i56i/rows.csv?accessType=DOWNLOAD' -o nypd.csv


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 2891M    0 2891M    0     0  5508k      0 --:--:--  0:08:57 --:--:-- 5693k


In [23]:
%%time
df = pd.read_csv('nypd.csv', low_memory = True, dtype='object')

CPU times: user 1min, sys: 5.92 s, total: 1min 6s
Wall time: 1min 6s


In [24]:
len(df)

8914838

In [25]:
df.query("KY_CD == '101'")

,CMPLNT_NUM,CMPLNT_FR_DT,CMPLNT_FR_TM,CMPLNT_TO_DT,CMPLNT_TO_TM,ADDR_PCT_CD,RPT_DT,KY_CD,OFNS_DESC,PD_CD,...,SUSP_SEX,TRANSIT_DISTRICT,Latitude,Longitude,Lat_Lon,PATROL_BORO,STATION_NAME,VIC_AGE_GROUP,VIC_RACE,VIC_SEX
0,25640826H10611,12/05/2006,10:38:00,NaN,(null),NaN,12/05/2006,101,MURDER & NON-NEGL. MANSLAUGHTER,NaN,...,(null),NaN,NaN,NaN,NaN,PATROL BORO QUEENS NORTH,(null),25-44,WHITE HISPANIC,M
1,16631501H8965,08/25/1973,12:15:00,NaN,(null),NaN,05/29/2006,101,MURDER & NON-NEGL. MANSLAUGHTER,NaN,...,(null),NaN,NaN,NaN,NaN,PATROL BORO MAN NORTH,(null),45-64,BLACK,F
2,11118212H8714,04/22/2006,16:50:00,NaN,(null),NaN,04/22/2006,101,MURDER & NON-NEGL. MANSLAUGHTER,NaN,...,(null),NaN,NaN,NaN,NaN,PATROL BORO BKLYN SOUTH,(null),45-64,BLACK,M
3,23471265H9399,07/20/2006,22:37:00,NaN,(null),NaN,07/20/2006,101,MURDER & NON-NEGL. MANSLAUGHTER,NaN,...,(null),NaN,NaN,NaN,NaN,PATROL BORO BKLYN SOUTH,(null),18-24,BLACK,M
4,24565927H10142,04/14/2006,02:37:00,NaN,(null),NaN,04/14/2006,101,MURDER & NON-NEGL. MANSLAUGHTER,NaN,...,(null),NaN,NaN,NaN,NaN,PATROL BORO BKLYN NORTH,(null),45-64,BLACK,M
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8912596,270206142H1,06/21/2023,16:25:00,NaN,(null),NaN,06/21/2023,101,MURDER & NON-NEGL. MANSLAUGHTER,NaN,...,M,NaN,NaN,NaN,NaN,PATROL BORO MAN SOUTH,(null),25-44,BLACK,M
8913795,276327599H1,10/19/2023,12:02:00,NaN,(null),NaN,10/19/2023,101,MURDER & NON-NEGL. MANSLAUGHTER,NaN,...,M,NaN,NaN,NaN,NaN,PATROL BORO QUEENS SOUTH,(null),65+,ASIAN / PACIFIC ISLANDER,M
8913847,278433670H2,12/03/2023,04:50:00,NaN,(null),101,12/03/2023,101,MURDER & NON-NEGL. MANSLAUGHTER,NaN,...,M,NaN,40.599583,-73.75619,"(40.599583, -73.75619)",PATROL BORO QUEENS SOUTH,(null),<18,BLACK,M
8914235,278433670H1,12/03/2023,04:50:00,NaN,(null),101,12/03/2023,101,MURDER & NON-NEGL. MANSLAUGHTER,NaN,...,M,NaN,40.599583,-73.75619,"(40.599583, -73.75619)",PATROL BORO QUEENS SOUTH,(null),25-44,BLACK,F


In [26]:
df.replace(to_replace = '(null)', value=np.nan, inplace = True)

In [27]:
len(df)

8914838

In [28]:
df.query("KY_CD == '101'")

,CMPLNT_NUM,CMPLNT_FR_DT,CMPLNT_FR_TM,CMPLNT_TO_DT,CMPLNT_TO_TM,ADDR_PCT_CD,RPT_DT,KY_CD,OFNS_DESC,PD_CD,...,SUSP_SEX,TRANSIT_DISTRICT,Latitude,Longitude,Lat_Lon,PATROL_BORO,STATION_NAME,VIC_AGE_GROUP,VIC_RACE,VIC_SEX
0,25640826H10611,12/05/2006,10:38:00,NaN,NaN,NaN,12/05/2006,101,MURDER & NON-NEGL. MANSLAUGHTER,NaN,...,NaN,NaN,NaN,NaN,NaN,PATROL BORO QUEENS NORTH,NaN,25-44,WHITE HISPANIC,M
1,16631501H8965,08/25/1973,12:15:00,NaN,NaN,NaN,05/29/2006,101,MURDER & NON-NEGL. MANSLAUGHTER,NaN,...,NaN,NaN,NaN,NaN,NaN,PATROL BORO MAN NORTH,NaN,45-64,BLACK,F
2,11118212H8714,04/22/2006,16:50:00,NaN,NaN,NaN,04/22/2006,101,MURDER & NON-NEGL. MANSLAUGHTER,NaN,...,NaN,NaN,NaN,NaN,NaN,PATROL BORO BKLYN SOUTH,NaN,45-64,BLACK,M
3,23471265H9399,07/20/2006,22:37:00,NaN,NaN,NaN,07/20/2006,101,MURDER & NON-NEGL. MANSLAUGHTER,NaN,...,NaN,NaN,NaN,NaN,NaN,PATROL BORO BKLYN SOUTH,NaN,18-24,BLACK,M
4,24565927H10142,04/14/2006,02:37:00,NaN,NaN,NaN,04/14/2006,101,MURDER & NON-NEGL. MANSLAUGHTER,NaN,...,NaN,NaN,NaN,NaN,NaN,PATROL BORO BKLYN NORTH,NaN,45-64,BLACK,M
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8912596,270206142H1,06/21/2023,16:25:00,NaN,NaN,NaN,06/21/2023,101,MURDER & NON-NEGL. MANSLAUGHTER,NaN,...,M,NaN,NaN,NaN,NaN,PATROL BORO MAN SOUTH,NaN,25-44,BLACK,M
8913795,276327599H1,10/19/2023,12:02:00,NaN,NaN,NaN,10/19/2023,101,MURDER & NON-NEGL. MANSLAUGHTER,NaN,...,M,NaN,NaN,NaN,NaN,PATROL BORO QUEENS SOUTH,NaN,65+,ASIAN / PACIFIC ISLANDER,M
8913847,278433670H2,12/03/2023,04:50:00,NaN,NaN,101,12/03/2023,101,MURDER & NON-NEGL. MANSLAUGHTER,NaN,...,M,NaN,40.599583,-73.75619,"(40.599583, -73.75619)",PATROL BORO QUEENS SOUTH,NaN,<18,BLACK,M
8914235,278433670H1,12/03/2023,04:50:00,NaN,NaN,101,12/03/2023,101,MURDER & NON-NEGL. MANSLAUGHTER,NaN,...,M,NaN,40.599583,-73.75619,"(40.599583, -73.75619)",PATROL BORO QUEENS SOUTH,NaN,25-44,BLACK,F


In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8914838 entries, 0 to 8914837
Data columns (total 35 columns):
 #   Column             Dtype 
---  ------             ----- 
 0   CMPLNT_NUM         object
 1   CMPLNT_FR_DT       object
 2   CMPLNT_FR_TM       object
 3   CMPLNT_TO_DT       object
 4   CMPLNT_TO_TM       object
 5   ADDR_PCT_CD        object
 6   RPT_DT             object
 7   KY_CD              object
 8   OFNS_DESC          object
 9   PD_CD              object
 10  PD_DESC            object
 11  CRM_ATPT_CPTD_CD   object
 12  LAW_CAT_CD         object
 13  BORO_NM            object
 14  LOC_OF_OCCUR_DESC  object
 15  PREM_TYP_DESC      object
 16  JURIS_DESC         object
 17  JURISDICTION_CODE  object
 18  PARKS_NM           object
 19  HADEVELOPT         object
 20  HOUSING_PSA        object
 21  X_COORD_CD         object
 22  Y_COORD_CD         object
 23  SUSP_AGE_GROUP     object
 24  SUSP_RACE          object
 25  SUSP_SEX           object
 26  TRANSIT_DISTRI

## Data Cleaning

In [29]:
# These columns are redundant
to_drop = ['Lat_Lon','X_COORD_CD','Y_COORD_CD']
# We created the CMPLNT_FR and CMPLNT_TO columns
# We have the longitude and latitude so the other coordinates are not needed
df = df.drop(to_drop, axis='columns')

###  CMPLNT_NUM         object   

In [33]:
# Remove any non-numeric characters from the CMPLNT_NUM attribute
df['CMPLNT_NUM'] = df['CMPLNT_NUM'].str.replace(r'\D', '', regex=True)

In [143]:
df['CMPLNT_NUM'] = pd.to_numeric(df['CMPLNT_NUM'], errors="coerce")
df['CMPLNT_NUM'] = np.abs(df['CMPLNT_NUM'].astype('int32'))
df = df[~df['CMPLNT_NUM'].isna()]
# Drop cases with duplicated complaint numbers
key_cnt = df['CMPLNT_NUM'].value_counts()
key_cnt [ key_cnt>1 ]
df = df[ ~df['CMPLNT_NUM'].isin( key_cnt [ key_cnt>1 ].index.values ) ]

In [144]:
len(df)

8841059

### CMPLNT_FR_DT       object
### CMPLNT_FR_TM       object
### CMPLNT_TO_DT       object
### CMPLNT_TO_TM       object

In [42]:
# There are a few rows that contain year 1015, 1016, ... that trigger an error during date conversion
# We replace all years written as 10XX with 20XX
# Note the usage of regular expressions
df.CMPLNT_FR_DT.replace(to_replace = '(\d\d)/(\d\d)/10(\d\d)', value=r'\1/\2/20\3', inplace = True, regex=True )
df.CMPLNT_TO_DT.replace(to_replace = '(\d\d)/(\d\d)/10(\d\d)', value=r'\1/\2/20\3', inplace = True, regex=True )

# Similarly, a few hours are written as 24:00:00, which also triggers errors.
# We fix these hours
df.CMPLNT_FR_TM.replace(to_replace = '24:00:00', value='00:00:00', inplace = True)
df.CMPLNT_TO_TM.replace(to_replace = '24:00:00', value='00:00:00', inplace = True)

# Convert the two separate date and time columns into single datetime columns
df['CMPLNT_FR'] = pd.to_datetime(df.CMPLNT_FR_DT + ' ' + df.CMPLNT_FR_TM, format='%m/%d/%Y %H:%M:%S', cache=True, errors="coerce")
df['CMPLNT_TO'] = pd.to_datetime(df.CMPLNT_TO_DT + ' ' + df.CMPLNT_TO_TM, format='%m/%d/%Y %H:%M:%S', cache=True, errors="coerce")

# These columns are redundant
to_drop = ['CMPLNT_FR_DT','CMPLNT_TO_DT','CMPLNT_FR_TM','CMPLNT_TO_TM']
# We created the CMPLNT_FR and CMPLNT_TO columns
# We have the longitude and latitude so the other coordinates are not needed
df = df.drop(to_drop, axis='columns')

In [43]:
df.CMPLNT_FR.isnull().sum()

702

In [44]:
df.CMPLNT_TO.isnull().sum()

1819707

In [45]:
df = df [ ~df.CMPLNT_FR.isnull() ]

###  ADDR_PCT_CD        object

In [46]:

df.ADDR_PCT_CD.replace(to_replace = '-99', value='99', inplace = True)
# df = df [ ~df.ADDR_PCT_CD.isnull() ]
df.ADDR_PCT_CD = pd.Categorical(df.ADDR_PCT_CD)

###  RPT_DT             object

In [47]:


# Convert RPT_DT to date
df.RPT_DT = pd.to_datetime(df.RPT_DT, format="%m/%d/%Y", cache=True)

,CMPLNT_NUM,ADDR_PCT_CD,RPT_DT,KY_CD,OFNS_DESC,PD_CD,PD_DESC,CRM_ATPT_CPTD_CD,LAW_CAT_CD,BORO_NM,...,TRANSIT_DISTRICT,Latitude,Longitude,PATROL_BORO,STATION_NAME,VIC_AGE_GROUP,VIC_RACE,VIC_SEX,CMPLNT_FR,CMPLNT_TO


### 7   KY_CD              object
### 8   OFNS_DESC          object

In [48]:
df.KY_CD.value_counts()

KY_CD
341    1556642
578    1187258
344     935220
109     783051
351     709197
        ...   
460         16
357         13
123          7
362          5
577          3
Name: count, Length: 75, dtype: int64

In [49]:
df.OFNS_DESC.replace(to_replace = 'KIDNAPPING', value='KIDNAPPING & RELATED OFFENSES', inplace = True)
df.OFNS_DESC.replace(to_replace = 'KIDNAPPING AND RELATED OFFENSES', value='KIDNAPPING & RELATED OFFENSES', inplace = True)

df.OFNS_DESC.replace(to_replace = 'AGRICULTURE & MRKTS LAW-UNCLASSIFIED', value='OTHER STATE LAWS (NON PENAL LAW)', inplace = True)
df.OFNS_DESC.replace(to_replace = 'OTHER STATE LAWS (NON PENAL LA', value='OTHER STATE LAWS (NON PENAL LAW)', inplace = True)

df.OFNS_DESC.replace(to_replace = 'ENDAN WELFARE INCOMP', value='OFFENSES RELATED TO CHILDREN', inplace = True)

df.OFNS_DESC.replace(to_replace = 'THEFT OF SERVICES', value='OTHER OFFENSES RELATED TO THEF', inplace = True)

df.OFNS_DESC.replace(to_replace = 'NYS LAWS-UNCLASSIFIED VIOLATION', value='OTHER STATE LAWS', inplace = True)

df.OFNS_DESC.replace(to_replace = 'FELONY SEX CRIMES', value='SEX CRIMES', inplace = True)

df.loc[df.KY_CD=='120','OFNS_DESC'] ='CHILD ABANDONMENT/NON SUPPORT'

df.loc[df.KY_CD=='125','OFNS_DESC'] ='NYS LAWS-UNCLASSIFIED FELONY'

offenses = df[ ["KY_CD", "OFNS_DESC"] ].drop_duplicates().dropna()
offenses['KY_CD'] = pd.Categorical(pd.to_numeric(offenses['KY_CD'] ).astype(int))
offenses = offenses.set_index("KY_CD")
offenses = offenses.sort_index()
offenses = offenses.reset_index()
offenses


,KY_CD,OFNS_DESC
0,101,MURDER & NON-NEGL. MANSLAUGHTER
1,102,HOMICIDE-NEGLIGENT-VEHICLE
2,103,"HOMICIDE-NEGLIGENT,UNCLASSIFIE"
3,104,RAPE
4,105,ROBBERY
...,...,...
70,676,NEW YORK CITY HEALTH CODE
71,677,OTHER STATE LAWS
72,678,MISCELLANEOUS PENAL LAW
73,685,ADMINISTRATIVE CODES


In [50]:
df.KY_CD = pd.Categorical(df.KY_CD)

In [51]:
df = df.drop('OFNS_DESC', axis='columns')

### 9   PD_CD              object
### 10  PD_DESC            object

In [52]:



df.loc[df.PD_CD=='694','PD_DESC'] ='INCEST'

df.loc[df.PD_CD=='234','PD_DESC'] ='BURGLARY,UNKNOWN TIME'

internal = df[ ["PD_CD", "PD_DESC"] ].drop_duplicates().dropna()
internal['PD_CD'] = pd.Categorical(pd.to_numeric(internal['PD_CD'] ).astype(int))
internal = internal.set_index("PD_CD")
internal = internal.sort_index()
internal = internal.reset_index()
internal

,PD_CD,PD_DESC
0,100,STALKING COMMIT SEX OFFENSE
1,101,ASSAULT 3
2,102,ASSAULT SCHOOL SAFETY AGENT
3,103,ASSAULT TRAFFIC AGENT
4,104,VEHICULAR ASSAULT (INTOX DRIVE
...,...,...
437,917,LEAVING THE SCENE OF AN ACCIDENT (SPI)
438,918,RECKLESS DRIVING
439,922,"TRAFFIC,UNCLASSIFIED MISDEMEAN"
440,969,"TRAFFIC,UNCLASSIFIED INFRACTIO"


In [53]:
df.PD_CD.isnull().sum()

7593

In [ ]:
# df = df[~df.PD_CD.isnull()]

In [54]:
df.PD_CD = pd.Categorical(df.PD_CD)

In [55]:
df = df.drop('PD_DESC', axis='columns')

### 11  CRM_ATPT_CPTD_CD   object

In [56]:
df.CRM_ATPT_CPTD_CD.value_counts()

CRM_ATPT_CPTD_CD
COMPLETED    8763338
ATTEMPTED     148422
Name: count, dtype: int64

In [57]:
df.CRM_ATPT_CPTD_CD = pd.Categorical(df.CRM_ATPT_CPTD_CD)

In [58]:
df.CRM_ATPT_CPTD_CD.isnull().sum()

168

In [59]:
df = df [ ~df.CRM_ATPT_CPTD_CD.isnull() ]


### 12  LAW_CAT_CD         object

In [60]:
df.LAW_CAT_CD.value_counts()

LAW_CAT_CD
MISDEMEANOR    4915109
FELONY         2791112
VIOLATION      1205539
Name: count, dtype: int64

In [61]:
df.LAW_CAT_CD = pd.Categorical(df.LAW_CAT_CD)

### 16  JURIS_DESC         object
### 17  JURISDICTION_CODE  object

In [62]:
df.JURISDICTION_CODE.isnull().sum()

0

In [63]:
# df = df[ ~df.JURISDICTION_CODE.isnull() ]

jusridiction = df[ ["JURISDICTION_CODE", "JURIS_DESC", ] ].drop_duplicates().dropna()
jusridiction['JURISDICTION_CODE'] = pd.to_numeric(jusridiction['JURISDICTION_CODE'] )
jusridiction['JURISDICTION_CODE'] = jusridiction['JURISDICTION_CODE'].astype(int)
jusridiction = jusridiction.set_index("JURISDICTION_CODE")
jusridiction = jusridiction.sort_index()
jusridiction = jusridiction.reset_index()
jusridiction

,JURISDICTION_CODE,JURIS_DESC
0,0,N.Y. POLICE DEPT
1,1,N.Y. TRANSIT POLICE
2,2,N.Y. HOUSING POLICE
3,3,PORT AUTHORITY
4,4,TRI-BORO BRDG TUNNL
5,6,LONG ISLAND RAILRD
6,7,AMTRACK
7,8,CONRAIL
8,9,STATN IS RAPID TRANS
9,11,N.Y. STATE POLICE


In [64]:
df.JURISDICTION_CODE = pd.Categorical(df.JURISDICTION_CODE)


In [65]:
df = df.drop('JURIS_DESC', axis='columns')

###  13  BORO_NM            object

In [66]:
df.BORO_NM.value_counts()

BORO_NM
BROOKLYN         2617509
MANHATTAN        2148821
BRONX            1928040
QUEENS           1800014
STATEN ISLAND     409551
Name: count, dtype: int64

In [ ]:
# df.BORO_NM.replace(to_replace = '(null)', value=np.nan, inplace = True)

In [67]:
df.BORO_NM.isnull().sum()

7825

In [68]:
df = df[~df.BORO_NM.isnull()]

In [69]:
df.BORO_NM = pd.Categorical(df.BORO_NM)

### 23  SUSP_AGE_GROUP     object
### 32  VIC_AGE_GROUP      object

In [70]:
# Both columns have a lot of noisy entries. We keep only the dominant groups, and also define an order
df.SUSP_AGE_GROUP = pd.Categorical(df.SUSP_AGE_GROUP, ordered=True, categories=['<18', '18-24',  '25-44', '45-64', '65+'])
df.VIC_AGE_GROUP = pd.Categorical(df.VIC_AGE_GROUP, ordered=True, categories=['<18', '18-24',  '25-44', '45-64', '65+'])


### 24  SUSP_RACE          object
### 25  SUSP_SEX           object

### 33  VIC_RACE           object
### 34  VIC_SEX            object

In [71]:
df.VIC_SEX.isnull().sum()

305

In [72]:
df.VIC_SEX.value_counts()

VIC_SEX
F    3480495
M    2966705
E    1291412
D    1158952
L       6062
U          4
Name: count, dtype: int64

In [73]:
df = df[~df.VIC_SEX.isnull()]

df.VIC_SEX.replace(to_replace = 'U', value=np.nan, inplace = True)

In [74]:
df.VIC_RACE.isnull().sum()

265

In [75]:
df.VIC_RACE.value_counts()

VIC_RACE
UNKNOWN                           2891284
BLACK                             2151130
WHITE                             1489355
WHITE HISPANIC                    1464952
ASIAN / PACIFIC ISLANDER           550638
BLACK HISPANIC                     316542
AMERICAN INDIAN/ALASKAN NATIVE      39433
OTHER                                  31
Name: count, dtype: int64

In [76]:
df.VIC_RACE.replace(to_replace = 'OTHER', value='UNKNOWN', inplace = True)

In [77]:
df.VIC_RACE.replace(to_replace = np.nan, value='UNKNOWN', inplace = True)

In [ ]:
# df = df[~df.VIC_RACE.isnull()]

In [78]:
df.SUSP_SEX.value_counts()

SUSP_SEX
M    3136392
U     996041
F     958789
Name: count, dtype: int64

In [79]:
# U is unknown, same is NULL.
df.SUSP_SEX.replace(to_replace = 'U', value=np.nan, inplace = True)
df.SUSP_SEX.replace(to_replace = '(null)', value=np.nan, inplace = True)

In [80]:
df.SUSP_RACE.value_counts()

SUSP_RACE
BLACK                             1923565
UNKNOWN                           1426964
WHITE HISPANIC                     874931
WHITE                              541187
BLACK HISPANIC                     270534
ASIAN / PACIFIC ISLANDER           172272
AMERICAN INDIAN/ALASKAN NATIVE      14752
OTHER                                  11
Name: count, dtype: int64

In [81]:
# Very small amount of OTHER values
df.SUSP_RACE.replace(to_replace = 'OTHER', value='UNKNOWN', inplace = True)
df.SUSP_RACE.replace(to_replace = np.nan, value='UNKNOWN', inplace = True)


In [82]:
df.SUSP_RACE = pd.Categorical(df.SUSP_RACE)
df.SUSP_SEX = pd.Categorical(df.SUSP_SEX)
df.VIC_RACE = pd.Categorical(df.VIC_RACE)
df.VIC_SEX = pd.Categorical(df.VIC_SEX)

In [83]:
df.dtypes

CMPLNT_NUM                    int32
ADDR_PCT_CD                category
RPT_DT               datetime64[ns]
KY_CD                      category
PD_CD                      category
CRM_ATPT_CPTD_CD           category
LAW_CAT_CD                 category
BORO_NM                    category
LOC_OF_OCCUR_DESC            object
PREM_TYP_DESC                object
JURISDICTION_CODE          category
PARKS_NM                     object
HADEVELOPT                   object
HOUSING_PSA                  object
SUSP_AGE_GROUP             category
SUSP_RACE                  category
SUSP_SEX                   category
TRANSIT_DISTRICT             object
Latitude                     object
Longitude                    object
PATROL_BORO                  object
STATION_NAME                 object
VIC_AGE_GROUP              category
VIC_RACE                   category
VIC_SEX                    category
CMPLNT_FR            datetime64[ns]
CMPLNT_TO            datetime64[ns]
dtype: object

###  14  LOC_OF_OCCUR_DESC  object

In [84]:
df.LOC_OF_OCCUR_DESC.value_counts()

LOC_OF_OCCUR_DESC
INSIDE         4560888
FRONT OF       2113038
OPPOSITE OF     225848
REAR OF         181727
OUTSIDE           4672
Name: count, dtype: int64

In [85]:
df.LOC_OF_OCCUR_DESC.replace(to_replace = '(null)', value=np.nan, inplace = True)

In [86]:
df.LOC_OF_OCCUR_DESC.isnull().sum()

1817457

In [87]:
df.LOC_OF_OCCUR_DESC = pd.Categorical(df.LOC_OF_OCCUR_DESC)

### Latitude                     object
### Longitude                    object

In [ ]:
# !sudo apt-get update
!sudo apt-get install python3-rtree
!sudo pip3 install geopandas descartes shapely ngram # matplotlib==3.1.3

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
python3-rtree is already the newest version (0.9.7-1).
0 upgraded, 0 newly installed, 0 to remove and 19 not upgraded.


In [88]:
import geopandas as gpd

In [89]:
df.Latitude = pd.to_numeric(df.Latitude, downcast='float')
df.Longitude  = pd.to_numeric(df.Longitude, downcast='float')

In [90]:
%%time
gdf = gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(df.Longitude, df.Latitude))

CPU times: user 10.2 s, sys: 1.96 s, total: 12.2 s
Wall time: 12 s


In [91]:
shapefile_url = 'https://data.cityofnewyork.us/api/geospatial/cpf4-rkhq?method=export&format=Shapefile'
df_nyc = gpd.GeoDataFrame.from_file(shapefile_url)

In [92]:
!mkdir -p maps
!curl -s https://s-media.nyc.gov/agencies/dcp/assets/files/zip/data-tools/bytes/nynta2010_23a.zip  -o maps/nynta2010_23a.zip
!cd maps && unzip -o nynta2010_23a.zip
shapefile = f"maps/nynta2010_23a/nynta2010.shp"
df_nyc = gpd.GeoDataFrame.from_file(shapefile)
df_nyc = df_nyc.to_crs(4326)

Archive:  nynta2010_23a.zip
  inflating: nynta2010_23a/nynta2010.shp  
  inflating: nynta2010_23a/nynta2010.dbf  
  inflating: nynta2010_23a/nynta2010.shx  
  inflating: nynta2010_23a/nynta2010.prj  
  inflating: nynta2010_23a/nynta2010.shp.xml  


In [93]:
df_nyc

,BoroCode,BoroName,CountyFIPS,NTACode,NTAName,Shape_Leng,Shape_Area,geometry
0,4,Queens,081,QN08,St. Albans,45401.316803,7.741275e+07,"POLYGON ((-73.75205 40.70523, -73.75174 40.704..."
1,2,Bronx,005,BX28,Van Cortlandt Village,21945.719299,2.566612e+07,"POLYGON ((-73.88705 40.88435, -73.88705 40.884..."
2,4,Queens,081,QN55,South Ozone Park,36708.169305,8.246139e+07,"POLYGON ((-73.80577 40.68293, -73.80552 40.682..."
3,3,Brooklyn,047,BK40,Windsor Terrace,19033.672066,1.404167e+07,"POLYGON ((-73.98017 40.66115, -73.98021 40.661..."
4,3,Brooklyn,047,BK50,Canarsie,43703.609666,8.208968e+07,"MULTIPOLYGON (((-73.88834 40.64671, -73.88835 ..."
...,...,...,...,...,...,...,...,...
190,2,Bronx,005,BX34,Melrose South-Mott Haven North,19589.196431,1.727176e+07,"POLYGON ((-73.90129 40.82048, -73.90160 40.819..."
191,2,Bronx,005,BX39,Mott Haven-Port Morris,35604.790810,4.189861e+07,"MULTIPOLYGON (((-73.89681 40.79581, -73.89694 ..."
192,2,Bronx,005,BX63,West Concourse,28571.879354,1.936642e+07,"POLYGON ((-73.91192 40.84326, -73.91195 40.843..."
193,5,Staten Island,085,SI22,West New Brighton-New Brighton-St. George,66052.593065,5.602857e+07,"POLYGON ((-74.07258 40.63794, -74.07330 40.637..."


In [94]:
%%time
# Match each accident with a neighborhood.
# Will take ~30 mins to run
# This is done with left join,
# so we preserve all the data points
# but we know which ones are not matching with the shapefile
gdf.crs = df_nyc.crs
gdf = gpd.sjoin(gdf, df_nyc, how='left')


CPU times: user 2min 10s, sys: 6.83 s, total: 2min 16s
Wall time: 2min 16s


In [95]:
gdf.dtypes

CMPLNT_NUM                    int32
ADDR_PCT_CD                category
RPT_DT               datetime64[ns]
KY_CD                      category
PD_CD                      category
CRM_ATPT_CPTD_CD           category
LAW_CAT_CD                 category
BORO_NM                    category
LOC_OF_OCCUR_DESC          category
PREM_TYP_DESC                object
JURISDICTION_CODE          category
PARKS_NM                     object
HADEVELOPT                   object
HOUSING_PSA                  object
SUSP_AGE_GROUP             category
SUSP_RACE                  category
SUSP_SEX                   category
TRANSIT_DISTRICT             object
Latitude                    float32
Longitude                   float32
PATROL_BORO                  object
STATION_NAME                 object
VIC_AGE_GROUP              category
VIC_RACE                   category
VIC_SEX                    category
CMPLNT_FR            datetime64[ns]
CMPLNT_TO            datetime64[ns]
geometry                   g

In [96]:
# We keep only boro_name and ntaname
todrop = [
    'index_right', 'BoroCode', 'CountyFIPS',
    'Shape_Area', 'Shape_Leng'
]

gdf = gdf.drop(todrop, axis='columns')

# Rename the columns
gdf = gdf.rename({
    'BoroName': 'BOROUGH',
    'NTAName': 'NEIGHBORHOOD',
    'NTACode': 'NEIGHBORHOOD_CODE',
}, axis='columns')

In [97]:
gdf['BOROUGH'] = gdf['BOROUGH'].str.upper()

In [98]:
# Mark as NULL all the lon/lat entries outside the NYC area
gdf.loc[gdf.BOROUGH.isnull(), 'Latitude'] = None
gdf.loc[gdf.BOROUGH.isnull(), 'Longitude'] = None

In [99]:
# Mark as NULL all the lon/lat entries that generate inconsistencies
mask = gdf.query('BOROUGH != BORO_NM and Latitude==Latitude and Longitude==Longitude').CMPLNT_NUM.values
condition = gdf.CMPLNT_NUM.isin(mask)

gdf.loc[condition, 'Latitude'] = None
gdf.loc[condition, 'Longitude'] = None

In [ ]:
# Delete the cases where the reported and detected boroughs are different
# to_delete = sorted(set(to_delete))
# gdf = gdf [ ~gdf.CMPLNT_NUM.isin(to_delete) ]

In [100]:
gdf = gdf.drop('geometry', axis='columns')

In [101]:
df = pd.DataFrame(gdf)

In [102]:
df.BORO_NM.value_counts()

BORO_NM
BROOKLYN         2617402
MANHATTAN        2148761
BRONX            1927965
QUEENS           1799962
STATEN ISLAND     409540
Name: count, dtype: int64

In [ ]:
#Temporarily, we drop these. We will add them back in the future
# df.drop(['BOROUGH','NEIGHBORHOOD','NEIGHBORHOOD_CODE'], axis='columns', inplace=True)

In [103]:
df = df[df.BOROUGH == df.BORO_NM]

In [104]:
df.drop(['BOROUGH'], axis='columns', inplace=True)
df.NEIGHBORHOOD_CODE = pd.Categorical(df.NEIGHBORHOOD_CODE)
df.NEIGHBORHOOD = pd.Categorical(df.NEIGHBORHOOD)

### TRANSIT_DISTRICT

In [105]:
df.TRANSIT_DISTRICT.value_counts()


TRANSIT_DISTRICT
4     31451
2     24022
1     19622
3     19010
33    18343
20    18086
12    14933
11    13997
32    13471
30    12562
34     8790
23     3174
Name: count, dtype: int64

In [106]:
len(df) - df.TRANSIT_DISTRICT.isnull().sum()

197461

In [107]:
df.drop('TRANSIT_DISTRICT', axis='columns', inplace=True)


### PREM_TYP_DESC

In [108]:
df.PREM_TYP_DESC.value_counts()

PREM_TYP_DESC
STREET                        2787962
RESIDENCE - APT. HOUSE        1901512
RESIDENCE-HOUSE                869009
RESIDENCE - PUBLIC HOUSING     651955
CHAIN STORE                    242605
                               ...   
CLOTHING BOUTIQUE                   4
SHOE STORE                          4
DOCTOR/DENTIST                      2
SCHOOL                              1
CHECK CASH                          1
Name: count, Length: 93, dtype: int64

In [109]:
df.PREM_TYP_DESC.isnull().sum()

38263

In [110]:
df = df [~df.PREM_TYP_DESC.isnull()]

In [111]:
df.PREM_TYP_DESC = pd.Categorical(df.PREM_TYP_DESC)

In [112]:
df.PARKS_NM.value_counts()

PARKS_NM
CENTRAL PARK                      2300
FLUSHING MEADOWS CORONA PARK      1953
WASHINGTON SQUARE PARK            1552
CONEY ISLAND BEACH & BOARDWALK    1428
RIVERSIDE PARK                     753
                                  ... 
ARSENAL NORTH                        1
UNNAMED PARK ON KEAP STREET          1
SPORTS PARK                          1
GRAND ARMY PLAZA MANHATTAN           1
GARDEN AT BEACH 36TH STREET          1
Name: count, Length: 1276, dtype: int64

In [113]:
df.PARKS_NM.value_counts().sum()

37152

In [114]:
df.drop('PARKS_NM', axis='columns', inplace=True)



 19  HADEVELOPT         object


In [115]:
df.HADEVELOPT.value_counts()

HADEVELOPT
INGERSOLL                                4553
WALD                                     2655
WILLIAMSBURG                             2457
NOSTRAND                                 2365
RIIS                                     2026
MARLBORO                                 1952
GRANT                                    1911
MANHATTANVILLE                           1910
SHEEPSHEAD BAY                           1719
WHITMAN                                  1415
WOODSIDE                                 1216
RIIS II                                  1121
SMITH                                    1095
RED HOOK EAST                             902
MARKHAM GARDENS                           624
WEST BRIGHTON I                           473
GLENWOOD                                  406
CAMPOS PLAZA I                            400
RED HOOK WEST                             234
LOWER EAST SIDE REHAB (GROUP 5)           195
FIRST HOUSES                              141
WEST BRIGHTON II       

In [116]:
df.drop('HADEVELOPT', axis='columns', inplace=True)


 20  HOUSING_PSA        object



In [117]:
df.HOUSING_PSA.value_counts()

HOUSING_PSA
670      9244
887      9169
720      7985
845      7960
632      7586
         ... 
32388       1
14168       1
5061        1
26215       1
8455        1
Name: count, Length: 4625, dtype: int64

In [118]:
df.HOUSING_PSA.value_counts().sum()

665380

In [119]:
df.drop('HOUSING_PSA', axis='columns', inplace=True)

 30  PATROL_BORO        object


In [120]:
df.PATROL_BORO.value_counts()

PATROL_BORO
PATROL BORO BRONX            1901409
PATROL BORO BKLYN SOUTH      1307331
PATROL BORO BKLYN NORTH      1296858
PATROL BORO MAN SOUTH        1082775
PATROL BORO MAN NORTH        1055502
PATROL BORO QUEENS NORTH      930470
PATROL BORO QUEENS SOUTH      858884
PATROL BORO STATEN ISLAND     407844
Name: count, dtype: int64

In [121]:
df.PATROL_BORO = pd.Categorical(df.PATROL_BORO)

In [122]:
df.PATROL_BORO.isnull().sum()

159

In [123]:
df = df[~df.PATROL_BORO.isnull()]

 31  STATION_NAME       object

In [124]:
df.STATION_NAME.value_counts()

STATION_NAME
125 STREET                        9163
14 STREET                         5100
42 ST.-PORT AUTHORITY BUS TERM    4772
34 ST.-PENN STATION               4358
42 ST.-TIMES SQUARE               3888
                                  ... 
DISTRICT 12 OFFICE                  16
DISTRICT 34 OFFICE                  15
OFF-SYSTEM                           8
WYCKOFF AVENUE                       6
DISTRICT 23 OFFICE                   6
Name: count, Length: 372, dtype: int64

In [125]:
df.STATION_NAME.replace(to_replace = '(null)', value=np.nan, inplace = True)

In [126]:
df.STATION_NAME.isnull().sum()



8645378

In [127]:
df.drop('STATION_NAME', axis='columns', inplace=True)

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8329781 entries, 17 to 8353048
Data columns (total 25 columns):
 #   Column             Dtype         
---  ------             -----         
 0   CMPLNT_NUM         int32         
 1   ADDR_PCT_CD        category      
 2   RPT_DT             datetime64[ns]
 3   KY_CD              category      
 4   PD_CD              category      
 5   CRM_ATPT_CPTD_CD   category      
 6   LAW_CAT_CD         category      
 7   BORO_NM            category      
 8   LOC_OF_OCCUR_DESC  category      
 9   PREM_TYP_DESC      category      
 10  JURISDICTION_CODE  category      
 11  SUSP_AGE_GROUP     category      
 12  SUSP_RACE          category      
 13  SUSP_SEX           category      
 14  Latitude           float32       
 15  Longitude          float32       
 16  PATROL_BORO        category      
 17  VIC_AGE_GROUP      category      
 18  VIC_RACE           category      
 19  VIC_SEX            category      
 20  CMPLNT_FR          date

## Data exploration

In this part we check the different values that appear in the columns. When we detect noisy results, we delete the corresponding values. In fact, many of the operations that are performed above, in the 'data cleaning' section, are the result of observations that we make here.

In [128]:
# Find the unique values in each column
#
# df.describe(include = [np.object, 'category']).T['unique']
unique = df.describe(include = 'all').T['unique'].sort_values()

In [129]:
unique

CRM_ATPT_CPTD_CD       2
SUSP_SEX               2
LAW_CAT_CD             3
VIC_SEX                5
BORO_NM                5
LOC_OF_OCCUR_DESC      5
VIC_AGE_GROUP          5
SUSP_AGE_GROUP         5
VIC_RACE               7
SUSP_RACE              7
PATROL_BORO            8
JURISDICTION_CODE     28
KY_CD                 75
ADDR_PCT_CD           77
PREM_TYP_DESC         93
NEIGHBORHOOD_CODE    195
NEIGHBORHOOD         195
PD_CD                442
CMPLNT_NUM           NaN
RPT_DT               NaN
Latitude             NaN
Longitude            NaN
CMPLNT_FR            NaN
CMPLNT_TO            NaN
Name: unique, dtype: object

In [130]:
for column in unique.index:
    if unique[column] < 200:
        print(df[column].value_counts())
        print("=====")

CRM_ATPT_CPTD_CD
COMPLETED    8693653
ATTEMPTED     147420
Name: count, dtype: int64
=====
SUSP_SEX
M    3113775
F     951834
Name: count, dtype: int64
=====
LAW_CAT_CD
MISDEMEANOR    4873147
FELONY         2769373
VIOLATION      1198553
Name: count, dtype: int64
=====
VIC_SEX
F    3463307
M    2948506
E    1276324
D    1147286
L       5649
Name: count, dtype: int64
=====
BORO_NM
BROOKLYN         2604188
MANHATTAN        2139491
BRONX            1901393
QUEENS           1788156
STATEN ISLAND     407845
Name: count, dtype: int64
=====
LOC_OF_OCCUR_DESC
INSIDE         4530462
FRONT OF       2100421
OPPOSITE OF     224720
REAR OF         180509
OUTSIDE           1420
Name: count, dtype: int64
=====
VIC_AGE_GROUP
25-44    2952572
45-64    1532029
18-24     896920
<18       414939
65+       326599
Name: count, dtype: int64
=====
SUSP_AGE_GROUP
25-44    1612683
18-24     600697
45-64     562255
<18       202302
65+        51595
Name: count, dtype: int64
=====
VIC_RACE
UNKNOWN                

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8350561 entries, 0 to 8353048
Data columns (total 22 columns):
 #   Column             Dtype         
---  ------             -----         
 0   CMPLNT_NUM         int32         
 1   ADDR_PCT_CD        category      
 2   RPT_DT             datetime64[ns]
 3   KY_CD              category      
 4   PD_CD              category      
 5   CRM_ATPT_CPTD_CD   category      
 6   LAW_CAT_CD         category      
 7   BORO_NM            category      
 8   LOC_OF_OCCUR_DESC  category      
 9   PREM_TYP_DESC      category      
 10  JURISDICTION_CODE  category      
 11  SUSP_AGE_GROUP     category      
 12  SUSP_RACE          category      
 13  SUSP_SEX           category      
 14  Latitude           float32       
 15  Longitude          float32       
 16  PATROL_BORO        category      
 17  VIC_AGE_GROUP      category      
 18  VIC_RACE           category      
 19  VIC_SEX            category      
 20  CMPLNT_FR          datet

In [131]:
df['PREM_TYP_DESC'].value_counts().count()

93

In [132]:
# All columns, except for the dates and spatial coordinates, are categorical
# Columns with less than a few thousand unique values are good candidates
# for ENUMs in the database given that the dataset is static.
# Also, in Pandas the internal representation becomes much more efficient
# as the Categoricals are stored as integers and not as strings
for column in unique.index:
    if column == 'RPT_DT':
        continue
    if df[column].value_counts().count() < 1000:
      df[column] = pd.Categorical(df[column])

In [ ]:
# With all the proper data typing the dataset went down in size from 1.9Gb+ to 425Mb.
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8329781 entries, 17 to 8353048
Data columns (total 25 columns):
 #   Column             Dtype         
---  ------             -----         
 0   CMPLNT_NUM         int32         
 1   ADDR_PCT_CD        category      
 2   RPT_DT             datetime64[ns]
 3   KY_CD              category      
 4   PD_CD              category      
 5   CRM_ATPT_CPTD_CD   category      
 6   LAW_CAT_CD         category      
 7   BORO_NM            category      
 8   LOC_OF_OCCUR_DESC  category      
 9   PREM_TYP_DESC      category      
 10  JURISDICTION_CODE  category      
 11  SUSP_AGE_GROUP     category      
 12  SUSP_RACE          category      
 13  SUSP_SEX           category      
 14  Latitude           float32       
 15  Longitude          float32       
 16  PATROL_BORO        category      
 17  VIC_AGE_GROUP      category      
 18  VIC_RACE           category      
 19  VIC_SEX            category      
 20  CMPLNT_FR          date

In [ ]:
df.memory_usage()

Index                66804488
CMPLNT_NUM           33402244
ADDR_PCT_CD           8353281
RPT_DT               66804488
KY_CD                 8353257
PD_CD                16721186
CRM_ATPT_CPTD_CD      8350693
LAW_CAT_CD            8350693
BORO_NM               8350773
LOC_OF_OCCUR_DESC     8350765
PREM_TYP_DESC         8353289
JURISDICTION_CODE     8351841
SUSP_AGE_GROUP        8350773
SUSP_RACE             8350917
SUSP_SEX              8350685
Latitude             33402244
Longitude            33402244
PATROL_BORO           8350933
VIC_AGE_GROUP         8350773
VIC_RACE              8350917
VIC_SEX               8350781
CMPLNT_FR            66804488
CMPLNT_TO            66804488
dtype: int64

In [ ]:
df.dtypes

CMPLNT_NUM                    int32
ADDR_PCT_CD                category
RPT_DT               datetime64[ns]
KY_CD                      category
PD_CD                      category
CRM_ATPT_CPTD_CD           category
LAW_CAT_CD                 category
BORO_NM                    category
LOC_OF_OCCUR_DESC          category
PREM_TYP_DESC              category
JURISDICTION_CODE          category
SUSP_AGE_GROUP             category
SUSP_RACE                  category
SUSP_SEX                   category
Latitude                    float32
Longitude                   float32
PATROL_BORO                category
VIC_AGE_GROUP              category
VIC_RACE                   category
VIC_SEX                    category
CMPLNT_FR            datetime64[ns]
CMPLNT_TO            datetime64[ns]
BOROUGH                    category
NEIGHBORHOOD_CODE          category
NEIGHBORHOOD               category
dtype: object

## Storing in a MySQL database

In [133]:
!sudo pip3 install -U -q PyMySQL sqlalchemy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.0/45.0 kB 730.4 kB/s eta 0:00:00


In [134]:
import os
from sqlalchemy import create_engine
from sqlalchemy import text

conn_string = 'mysql+pymysql://{user}:{password}@{host}/?charset=utf8mb4'.format(
    host = 'db.ipeirotis.org',
    user = 'root',
    password = mysql_pass,
    encoding = 'utf8mb4')

engine = create_engine(conn_string)


In [145]:
# Query to create a database
db_name = 'nypd'

sql = f"DROP DATABASE IF EXISTS {db_name}"
with engine.connect() as connection:
  connection.execute(text(sql))

# Create a database
sql = f"CREATE DATABASE IF NOT EXISTS {db_name} DEFAULT CHARACTER SET 'utf8mb4'"
with engine.connect() as connection:
  connection.execute(text(sql))


In [146]:

# And lets switch to the database
sql = f"USE {db_name}"
with engine.connect() as connection:
  connection.execute(text(sql))


In [147]:
df['NEIGHBORHOOD'] = df['NEIGHBORHOOD'].str.replace('\'', '’', regex=False)

In [148]:
NEIGHBORHOOD_enum = "ENUM('" + ("','".join(sorted(df.NEIGHBORHOOD.astype(str).unique()))) + "')"


In [149]:
print(NEIGHBORHOOD_enum)

ENUM('Airport','Allerton-Pelham Gardens','Annadale-Huguenot-Prince’s Bay-Eltingville','Arden Heights','Astoria','Auburndale','Baisley Park','Bath Beach','Battery Park City-Lower Manhattan','Bay Ridge','Bayside-Bayside Hills','Bedford','Bedford Park-Fordham North','Bellerose','Belmont','Bensonhurst East','Bensonhurst West','Borough Park','Breezy Point-Belle Harbor-Rockaway Park-Broad Channel','Briarwood-Jamaica Hills','Brighton Beach','Bronxdale','Brooklyn Heights-Cobble Hill','Brownsville','Bushwick North','Bushwick South','Cambria Heights','Canarsie','Carroll Gardens-Columbia Street-Red Hook','Central Harlem North-Polo Grounds','Central Harlem South','Charleston-Richmond Valley-Tottenville','Chinatown','Claremont-Bathgate','Clinton','Clinton Hill','Co-op City','College Point','Corona','Crotona Park East','Crown Heights North','Crown Heights South','Cypress Hills-City Line','DUMBO-Vinegar Hill-Downtown Brooklyn-Boerum Hill','Douglas Manor-Douglaston-Little Neck','Dyker Heights','East C

In [150]:
NCODE_enum = "ENUM('" + ("','".join(sorted(df.NEIGHBORHOOD_CODE.astype(str).unique()))) + "')"

In [151]:
# In principle, we can let Pandas create the table, but we want to be a bit more predise
# with the data types, and we want to add documentation for each column
# from https://data.cityofnewyork.us/Public-Safety/NYPD-Complaint-Data-Historic/qgea-i56i


sql = f'''
CREATE TABLE nypd (
  CMPLNT_NUM int,
  CMPLNT_FR datetime,
  CMPLNT_TO datetime,
  RPT_DT date,
  KY_CD SMALLINT,
  PD_CD SMALLINT,
  CRM_ATPT_CPTD_CD enum('COMPLETED','ATTEMPTED'),
  LAW_CAT_CD enum('FELONY','MISDEMEANOR','VIOLATION'),
  JURISDICTION_CODE SMALLINT,
  BORO_NM enum('BRONX','BROOKLYN','MANHATTAN','QUEENS','STATEN ISLAND'),
  NEIGHBORHOOD {NEIGHBORHOOD_enum},
  NEIGHBORHOOD_CODE {NCODE_enum},
  ADDR_PCT_CD SMALLINT,
  LOC_OF_OCCUR_DESC enum('FRONT OF','INSIDE','OPPOSITE OF','OUTSIDE','REAR OF'),
  PATROL_BORO enum('PATROL BORO BRONX', 'PATROL BORO BKLYN SOUTH','PATROL BORO BKLYN NORTH','PATROL BORO MAN SOUTH','PATROL BORO MAN NORTH','PATROL BORO QUEENS NORTH','PATROL BORO QUEENS SOUTH','PATROL BORO STATEN ISLAND'),
  PREM_TYP_DESC varchar(30),
  SUSP_RACE enum('UNKNOWN', 'BLACK', 'WHITE', 'WHITE HISPANIC', 'ASIAN / PACIFIC ISLANDER', 'BLACK HISPANIC', 'AMERICAN INDIAN/ALASKAN NATIVE'),
  VIC_RACE enum('UNKNOWN', 'BLACK', 'WHITE', 'WHITE HISPANIC', 'ASIAN / PACIFIC ISLANDER', 'BLACK HISPANIC', 'AMERICAN INDIAN/ALASKAN NATIVE'),
  SUSP_AGE_GROUP enum('<18', '18-24',  '25-44', '45-64', '65+'),
  VIC_AGE_GROUP enum('<18', '18-24',  '25-44', '45-64', '65+'),
  SUSP_SEX enum('M', 'F'),
  VIC_SEX enum('M', 'F', 'E', 'D', 'L'),
  Latitude double,
  Longitude double,
  PRIMARY KEY (CMPLNT_NUM)
) ENGINE=InnoDB DEFAULT CHARSET=utf8mb4;
'''
with engine.connect() as connection:
  connection.execute(text(sql))

In [152]:
# Create a table
# See http://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.to_sql.html for the documentation
from tqdm import tqdm
batchsize = 50000
batches = len(df) // batchsize + 1

t = tqdm(range(batches))

for i in t:
    # print("Batch:",i)
    # continue # Cannot execute this on Travis
    start = batchsize * i
    end = batchsize * (i+1)
    df[start:end].to_sql(
        name = 'nypd',
        schema = db_name,
        con = engine,
        if_exists = 'append',
        index = False,
        chunksize = 1000)

100%|██████████| 177/177 [19:07<00:00,  6.48s/it]


In [153]:
sql = "CREATE INDEX ix_lat ON nypd.nypd(Latitude)"
with engine.connect() as connection:
  connection.execute(text(sql))

In [154]:
sql = "CREATE INDEX ix_lon ON nypd.nypd(Longitude)"
with engine.connect() as connection:
  connection.execute(text(sql))

In [155]:
sql = "CREATE INDEX ix_LAW_CAT_CD ON nypd.nypd(LAW_CAT_CD)"
with engine.connect() as connection:
  connection.execute(text(sql))

In [156]:
sql = "CREATE INDEX ix_BORO_NM ON nypd.nypd(BORO_NM)"
with engine.connect() as connection:
  connection.execute(text(sql))

In [157]:
sql = "CREATE INDEX ix_KY_CD ON nypd.nypd(KY_CD)"
with engine.connect() as connection:
  connection.execute(text(sql))

In [158]:
sql = "CREATE INDEX ix_RPT_DT ON nypd.nypd(RPT_DT)"
with engine.connect() as connection:
  connection.execute(text(sql))

In [159]:
sql = "CREATE INDEX ix_CMPLNT_FR ON nypd.nypd(CMPLNT_FR)"
with engine.connect() as connection:
  connection.execute(text(sql))

In [160]:
offenses = offenses[offenses.OFNS_DESC != "(null)"]

In [161]:
offenses

,KY_CD,OFNS_DESC
0,101,MURDER & NON-NEGL. MANSLAUGHTER
1,102,HOMICIDE-NEGLIGENT-VEHICLE
2,103,"HOMICIDE-NEGLIGENT,UNCLASSIFIE"
3,104,RAPE
4,105,ROBBERY
...,...,...
70,676,NEW YORK CITY HEALTH CODE
71,677,OTHER STATE LAWS
72,678,MISCELLANEOUS PENAL LAW
73,685,ADMINISTRATIVE CODES


In [162]:
sql = "DROP TABLE IF EXISTS offense_codes;"
with engine.connect() as connection:
  connection.execute(text(sql))

sql = '''
CREATE TABLE offense_codes (
  KY_CD smallint,
  OFNS_DESC varchar(32),
  PRIMARY KEY (KY_CD)
) ENGINE=InnoDB DEFAULT CHARSET=utf8mb4;
'''
with engine.connect() as connection:
  connection.execute(text(sql))

offenses.to_sql(
        name = 'offense_codes',
        schema = db_name,
        con = engine,
        if_exists = 'append',
        index = False)

75

In [163]:
sql = "DROP TABLE IF EXISTS jurisdiction_codes;"
with engine.connect() as connection:
  connection.execute(text(sql))

sql = '''
CREATE TABLE jurisdiction_codes (
  JURISDICTION_CODE smallint,
  JURIS_DESC varchar(40),
  PRIMARY KEY (JURISDICTION_CODE)
) ENGINE=InnoDB DEFAULT CHARSET=utf8mb4;
'''
with engine.connect() as connection:
  connection.execute(text(sql))


jusridiction.to_sql(
        name = 'jurisdiction_codes',
        schema = db_name,
        con = engine,
        if_exists = 'append',
        index = False)

28

In [164]:
internal.PD_DESC.str.len().max()


71

In [165]:
internal = internal.query("PD_DESC !=	'CRIMINAL DISPOSAL FIREARM 1 &' ")
internal = internal.query("PD_DESC !=	'UNFINSH FRAME 2' ")
internal = internal.query("PD_DESC !=	'WEAPONS POSSESSION 1 & 2' ")
internal = internal.query("PD_DESC !=	'CRIM POS WEAP 4' ")


In [166]:
internal

,PD_CD,PD_DESC
0,100,STALKING COMMIT SEX OFFENSE
1,101,ASSAULT 3
2,102,ASSAULT SCHOOL SAFETY AGENT
3,103,ASSAULT TRAFFIC AGENT
4,104,VEHICULAR ASSAULT (INTOX DRIVE
...,...,...
437,917,LEAVING THE SCENE OF AN ACCIDENT (SPI)
438,918,RECKLESS DRIVING
439,922,"TRAFFIC,UNCLASSIFIED MISDEMEAN"
440,969,"TRAFFIC,UNCLASSIFIED INFRACTIO"


In [167]:
sql = "DROP TABLE IF EXISTS penal_codes;"
with engine.connect() as connection:
  connection.execute(text(sql))

sql = '''
CREATE TABLE penal_codes (
  PD_CD smallint,
  PD_DESC varchar(80),
  PRIMARY KEY (PD_CD)
) ENGINE=InnoDB DEFAULT CHARSET=utf8mb4;
'''
with engine.connect() as connection:
  connection.execute(text(sql))


internal.to_sql(
        name = 'penal_codes',
        schema = db_name,
        con = engine,
        if_exists = 'append',
        index = False)

440

## TODO

### Add the penal code data as a separate table

`!curl 'https://data.cityofnewyork.us/api/views/qgea-i56i/files/65f25845-1551-4d21-91dc-869c977cd93d?download=true&filename=PDCode_PenalLaw.xlsx' -o PDCode_PenalLaw.xlsx`

### Examine whether to normalize

The fields

  
PREM_TYP_DESC    
HADEVELOPT    
PARKS_NM                     

would be better off as foreign keys or enums. They take too much space as strings.